In [38]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
import time
import numpy as np
import math
import pandas as pd
from PIL import Image
from datetime import datetime
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
import random
import datetime
import os

from sklearn import preprocessing 

In [39]:
os.chdir("../../../kaggleData/data")
os.getcwd()

'/home/ros/Documents/kaggleData/data'

In [40]:
train = pd.read_csv("train.csv")
lable = pd.read_csv("labels.csv")
test = pd.read_csv("sample_submission.csv")

In [41]:
lable_length = len(lable)import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
import time
import numpy as np
import math
import pandas as pd
from PIL import Image
from datetime import datetime
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
import random
import datetime
import os

from sklearn import preprocessing 
train_length = len(train)
test_length = len(test)
print(train_length)
print(lable_length)
print(test_length)

109237
1103
7443


In [42]:
np.array(lable)

array([[0, 'culture::abruzzi'],
       [1, 'culture::achaemenid'],
       [2, 'culture::aegean'],
       ...,
       [1100, 'tag::zeus'],
       [1101, 'tag::zigzag pattern'],
       [1102, 'tag::zodiac']], dtype=object)

In [43]:
np.array(test)

array([['10023b2cc4ed5f68', '0 1 2'],
       ['100fbe75ed8fd887', '0 1 2'],
       ['101b627524a04f19', '0 1 2'],
       ...,
       ['ffe54b454396d97c', '0 1 2'],
       ['ffe7d7db4e4aa37f', '0 1 2'],
       ['ffed0a4aca0d5457', '0 1 2']], dtype=object)

In [44]:
test.head

<bound method NDFrame.head of                     id attribute_ids
0     10023b2cc4ed5f68         0 1 2
1     100fbe75ed8fd887         0 1 2
2     101b627524a04f19         0 1 2
3     10234480c41284c6         0 1 2
4     1023b0e2636dcea8         0 1 2
5      1039cd6cf85845c         0 1 2
6      103a5b3f83fbe88         0 1 2
7     10413aaae8d6a9a2         0 1 2
8     10423822b93a65ab         0 1 2
9     1052bf702cb099f7         0 1 2
10     10543c918a43a8d         0 1 2
11    105c9a3453da79c3         0 1 2
12    1060688bbf6eac87         0 1 2
13    106a247caeabd15a         0 1 2
14    106e21606add59f3         0 1 2
15    107c38495881b6c9         0 1 2
16    108815dd3752ab64         0 1 2
17    10943defdd5d5e89         0 1 2
18    10a39a78c44ef27c         0 1 2
19     10ab70df067bdb4         0 1 2
20    10b28e3de3566582         0 1 2
21    10b32964331a6cc3         0 1 2
22    10b4562e7fa6f668         0 1 2
23    10db1c338e1d822f         0 1 2
24    10e0c215f5f3084e         0 1 2
25    10

In [45]:
np.array(train)

array([['1000483014d91860', '147 616 813'],
       ['1000fe2e667721fe', '51 616 734 813'],
       ['1001614cb89646ee', '776'],
       ...,
       ['ffff3e66a42ab868', '156 763'],
       ['ffff45b237a32bd5', '121 433'],
       ['ffffbf00586b8e37', '462 733 813 1020']], dtype=object)

In [46]:
def creatData(train,lable_length):
    train = np.array(train)
    train_data = []
    for t in range(train_length):
        v = np.zeros(lable_length)-1
        #print(train[t,1])
        for s in train[t,1].split(" "):
            #print(s)
            v[int(s)] = 1
        train_data.append([train[t,0],v])
    return np.array(train_data)

In [47]:
train_lib = creatData(train,lable_length)
print(train_lib)

[['1000483014d91860' array([-1., -1., -1., ..., -1., -1., -1.])]
 ['1000fe2e667721fe' array([-1., -1., -1., ..., -1., -1., -1.])]
 ['1001614cb89646ee' array([-1., -1., -1., ..., -1., -1., -1.])]
 ...
 ['ffff3e66a42ab868' array([-1., -1., -1., ..., -1., -1., -1.])]
 ['ffff45b237a32bd5' array([-1., -1., -1., ..., -1., -1., -1.])]
 ['ffffbf00586b8e37' array([-1., -1., -1., ..., -1., -1., -1.])]]


In [48]:
train_transformer = transforms.Compose([
  transforms.Resize((128,128)),              # resize the image to 
  #transforms.RandomHorizontalFlip(),  # randomly flip image horizontally
  transforms.ToTensor()])             # transform it into a PyTorch Tensor

AttributeError: module 'torchvision.transforms' has no attribute 'Resize'

In [ ]:
class trainDataset(Dataset):
    def __init__(self, train_lib, transform=None):
        self.filenames = train_lib[:,0]
        self.labels = train_lib[:,1]
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image = Image.open("train/"+format(self.filenames[idx])+'.png')  # PIL image
        image = self.transform(image)
        return image, self.labels[idx]


In [ ]:
class testDataset(Dataset):
    def __init__(self, test_lib, transform=None):
        test_lib = np.array(test_lib)
        self.filenames = test_lib[:,0]
        #self.labels = test_lib[:,1]
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image = Image.open("test/"+format(self.filenames[idx])+'.png')  # PIL image
        image = self.transform(image)
        return image,self.filenames[idx]

In [ ]:
train_dataloader = DataLoader(trainDataset(train_lib, train_transformer), 
                              batch_size=64, shuffle=True)

In [ ]:
test_dataloader = DataLoader(testDataset(test, train_transformer),batch_size=64,shuffle=True)

In [ ]:
np.array(test_dataloader.dataset[0][1]).shape

In [ ]:
(np.array(train_dataloader.dataset[4][0]).shape)

In [ ]:
resnet_model = models.resnet18(pretrained=False) 
resnet_model.fc= nn.Linear(in_features=512, out_features=lable_length)

In [ ]:
cnn = resnet_model
cnn.cuda()
print(cnn)

In [ ]:
def train(epoch):
    for step, (x, y) in enumerate(train_dataloader):
        data = Variable(x).cuda()   # batch x
        target = Variable(y).cuda()   # batch y
        #print(data.type())
        #print(target.type())
        output = cnn(data)               # cnn output
        #loss = nn.functional.nll_loss(output, target)
        loss = loss_func(output, target.float())   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
        if step==0:
            start = time.time()
            ti = 0
        elif step==100:
            ti = time.time()-start #total time = ti*(length/100)
            #print(ti)
            ti = ti*(len(train_dataloader)/100)
        if step % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime Remain : {} '.format(epoch, 
                                                                            step * len(data), 
                                                                            len(train_dataloader.dataset),
                                                                            100.*step/len(train_dataloader), 
                                                                            loss.data.item(),
                                                                            datetime.timedelta(seconds=(ti*(((len(train_dataloader)-step)/len(train_dataloader)))))))
            #print('    Time Remain : {}'.format(datetime.timedelta(seconds=(ti*(((length-i)/length))))))
    print("Finish")

In [ ]:
for epoch in range(4):
    optimizer = torch.optim.SGD(cnn.parameters(), lr=0.001/(2**epoch),momentum=0.9)
    #optimizer = torch.optim.ASGD(cnn.parameters(), lr=0.001)
    #optimizer = torch.optim.Adam(cnn.parameters(), lr=0.0001/(2**epoch))
    loss_func = torch.nn.MSELoss()
    #loss_func = torch.nn.MultiLabelMarginLoss()
    #loss_func = torch.nn.SmoothL1Loss()
    #loss_func = FocalLoss(class_num = lable_length)
    #optimizer = torch.optim.ASGD(cnn.parameters(), lr=0.0005/(epoch+1))
    train(epoch)
    if epoch==0:
        torch.save(cnn, '/home/ros/Documents/net_E0.pkl')
    elif epoch==1:
        torch.save(cnn, '/home/ros/Documents/net_E1.pkl')
    elif epoch==2:
        torch.save(cnn, '/home/ros/Documents/net_E2.pkl')
    elif epoch==3:
        torch.save(cnn, '/home/ros/Documents/net_E3.pkl')
        
    

In [ ]:
torch.save(cnn, '/home/ros/Documents/net.pkl')

In [ ]:
def testFromT(model):
    model = model.eval()
    #model = model.cpu().eval()
    ans = []
    for step, (x, y) in enumerate(train_dataloader):
        data = Variable(x).cuda()
        #data = Variable(x)
        target = np.array(y)
        output = model(data)
        v = output.cpu().detach()
        v = torch.sigmoid(v)
        #avg=torch.max(v,dim=1,keepdim=True)
        #print(len(v))
        #for i in range(len(v)):
            #v[i] = 1*(v[i]/avg[i])
            #v[i] = v[i]**2
        #vp = preprocessing.minmax_scale(v, feature_range=(-1,1),axis=1)
        vp = torch.sigmoid(v)
        v = np.array(v)
        vp = np.array(vp)
        if step==0:
            break
        #print(v)
        #v = preprocessing.minmax_scale(v, feature_range=(0,1),axis=1)
        #v = min_max_scaler.fit_transform(v)
#         v = torch.from_numpy(v)
#         v = F.softmax(v, dim=0)
#         v = np.array(v)
        #v = sigmoid(v)
        #print("==========")
        #print(np.max(v[0]))
        #print(np.min(v[0]))
        #print("==========")
    print("Finish")
    return v[0],vp[0],target[0]
    

In [ ]:
v,vp,t = testFromT(cnn)

In [ ]:
fig = plt.figure()
plt.plot(v[100:800], '-')
plt.plot(t[100:800], '--');

In [ ]:
def findPre(output):
    a = ''
    output = np.array(output)
    #output = np.where(output > 0.8,1,0)
    for i in range(len(output)):
        if output[i]>80:
            a = a + format(i)+' '
    #print(a)
    return a
    

In [ ]:
def test(model):
    model = model.eval()
    ans = []
    for step, (x, y) in enumerate(test_dataloader):
        data = Variable(x).cuda()
        target = y
        output = model(data)
        v = output.cpu().detach()
        
        v = np.array(v)
        v = preprocessing.minmax_scale(v, feature_range=(-100,100))
        #v = min_max_scaler.fit_transform(v)
#         v = torch.from_numpy(v)
#         v = F.softmax(v, dim=0)
#         v = np.array(v)
        #v = sigmoid(v)
        #print(np.max(v[0]))
        #print(np.min(v[0]))
        
        for i in range(len(v)):
            #V = (v[i]+abs(np.min(v[i])))/(abs(np.min(v[i]))+abs(np.max(v[i])))
            #print(v)
            s = findPre(v[i])
            ans.append([target[i],s])
        if step %10 == 0:
            print('[{}/{} ({:.0f}%)]'.format(step * len(data), 
                                        len(test_dataloader.dataset),
                                        100.*step/len(test_dataloader)))

    return ans
    print("Finish")

In [ ]:
sub = test(cnn)

In [ ]:
sub =  pd.DataFrame(sub)

In [ ]:
sub = sub.rename(index=str, columns={0: "id", 1: "attribute_ids"})

In [ ]:
sub.head

In [ ]:
sub.to_csv('submission.csv', index=False)